In [19]:
from tensorflow.keras.datasets import cifar10
import numpy as np
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [20]:
# 샘플링 비율 설정 (10%)
sampling_ratio = 0.1

# 훈련 데이터 샘플링
num_samples = int(len(X_train) * sampling_ratio)
indices = np.random.choice(len(X_train), num_samples, replace=False) # 비복원추출
X_train_sampled = X_train[indices]
y_train_sampled = y_train[indices]

In [21]:
# 테스트 데이터 샘플링
num_samples_test = int(len(X_test) * sampling_ratio)
indices_test = np.random.choice(len(X_test), num_samples_test, replace=False)
X_test_sampled = X_test[indices_test]
y_test_sampled = y_test[indices_test]
print("새로운 훈련 데이터 크기:", X_train_sampled.shape)
print("새로운 테스트 데이터 크기:", X_test_sampled.shape)

새로운 훈련 데이터 크기: (5000, 32, 32, 3)
새로운 테스트 데이터 크기: (1000, 32, 32, 3)


In [22]:
import tensorflow as tf
X_train_resized = tf.image.resize(X_train_sampled, [224, 224])
X_test_resized = tf.image.resize(X_test_sampled, [224, 224])

In [23]:
train_images = X_train_resized / 255.0
test_images = X_test_resized / 255.0

print(train_images.shape)
print(test_images.shape)

(5000, 224, 224, 3)
(1000, 224, 224, 3)


In [24]:
# 종속변수 확인
print(type(y_test_sampled))
print(y_test_sampled[:10])

# 변수명 변경
train_labels = y_train_sampled
test_labels = y_test_sampled

<class 'numpy.ndarray'>
[[0]
 [9]
 [7]
 [8]
 [1]
 [9]
 [1]
 [1]
 [0]
 [4]]


In [25]:
from keras import models
from keras import layers
model = models.Sequential()
# 합성곱 계층 1
model.add(layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), padding='valid',
input_shape=(224, 224, 3), activation='relu'))

# 합성곱 계층 2
model.add(layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='valid',activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))
# strides = 2인데, 풀링은 3x3을 하므로 겹치기(overlapping) 방식으로 풀링이 일어난다
#풀링 시 정보를 공유하려는 목적이지만 효과가 분명하지 않고, 계산이 복잡하여 이후에는 잘 사용되지 않았다

# 합성곱 계층 3
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='valid',activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))

# 합성곱 계층 4
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='valid',activation='relu'))

# 합성곱 계층 5
model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='valid',activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))

In [26]:
# Flattening
model.add(layers.Flatten())

# 완전연결층 1
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

# 완전연결층 2
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

# 완전연결층 3. Output Layer. AlexNet은 1,000 분류를 하였으나, CIFAR-10은 10분류이다
model.add(layers.Dense(10, activation='softmax'))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 conv2d_6 (Conv2D)           (None, 50, 50, 256)       614656    
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 24, 24, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 22, 22, 384)       885120    
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 10, 10, 384)       0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 8, 384)        

In [ ]:
# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

# 모델 훈련
# 데이터와 훈련량이 충분하지 않기 때문에 성능은 좋지 않다
history = model.fit(train_images, train_labels, epochs=10, batch_size=128,validation_data=(test_images, test_labels))

Epoch 1/10
13/40 [========>.....................] - ETA: 13:48 - loss: 2.3708 - acc: 0.0992

In [ ]:
# 데이터와 훈련량이 충분하지 않기 때문에 성능은 좋지 않다
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
print('Accuracy of each epoch:', acc)
epochs = range(1, len(acc) +1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
# 랜덤 한 개 이미지 추출
import numpy as np
sample = np.random.choice(np.arange(0, len(test_images)))
print(sample)
print(test_labels[sample])

In [ ]:
# 모델을 사용해 예측
predictions = model.predict(test_images[sample:sample+1])

# 가장 큰 확률의 인덱스
predicted_class = np.argmax(predictions, axis=1)[0]
print("예측된 숫자:", predicted_class)

# 클래스별 확률
print("\n클래스별 확률:")
for i, prob in enumerate(predictions[0]):
  print(f"Class {i}: {prob:.4f}")